In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
sys.path.append('/home/jupyter-will/repos/cigarmath')
import cigarmath as cm

In [2]:
sites = {'barcode':(6407,6440),
         'Psi-Target': (1165, 1191),
         'Gag-Target': (1755,1781)}

In [3]:
from collections import Counter, defaultdict
from typing import Tuple, List

def get_combined_segments(path):
    
    sorted_segments = sorted(cm.io.segment_stream_pysam(path, mode='rb'), key = lambda x: x.query_name)
    return list(cm.io.combined_segment_stream(sorted_segments))


def detect_deletions(combined_stream):
    del_blocks = Counter()

    for start, cigartuples, segments in combined_stream:
        if cigartuples:
            deletion_blocks = list(cm.reference_deletion_blocks(cigartuples, reference_start=start, min_size=50))
            for block in deletion_blocks:
                del_blocks[block] += 1
    
    return del_blocks    
    
    
def merge_blocks(blocks: Counter, n: int, *, sum_counts: bool = True) -> Counter:
    """
    Merge blocks whose start or stop are within n units of each other.
    
    Args:
        blocks: Counter keyed by (start, stop) tuples.
        n: integer proximity threshold.
        sum_counts: if True, representative's count is sum of component counts.
                    if False, representative's count is the original count of the chosen block.
    
    Returns:
        Counter of merged blocks: representative_block -> count
    """
    # Early exit
    if not blocks:
        return Counter()
    
    items: List[Tuple[Tuple[int,int], int]] = list(blocks.items())  # [((s,e), cnt), ...]
    m = len(items)
    
    # Union-find
    parent = list(range(m))
    rank = [0]*m
    def find(a):
        while parent[a] != a:
            parent[a] = parent[parent[a]]
            a = parent[a]
        return a
    def union(a,b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[rb] < rank[ra]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1
    
    # Connect indices if starts within n or stops within n
    # O(m^2) worst-case; if m is large you can optimize by sorting and sweeping.
    for i in range(m):
        (si, ei), _ = items[i]
        for j in range(i+1, m):
            (sj, ej), _ = items[j]
            if abs(si - sj) <= n or abs(ei - ej) <= n:
                union(i, j)
    
    # Group by root
    groups = defaultdict(list)
    for idx in range(m):
        groups[find(idx)].append(idx)
    
    merged = Counter()
    for group_idxs in groups.values():
        # Choose representative: highest original count, tie-break by smaller start then smaller stop
        rep_idx = max(
            group_idxs,
            key=lambda idx: (items[idx][1], -items[idx][0][0]*0 + -items[idx][0][1]*0)  # primary: count
        )
        # better tie-breaking explicitly:
        # rep_idx = max(group_idxs, key=lambda idx: (items[idx][1], -items[idx][0][0], -items[idx][0][1]))
        # But above uses negative start/stop to prefer smaller values after equal counts.
        # (I keep the explicit version below)
        rep_idx = max(group_idxs, key=lambda idx: (items[idx][1], -items[idx][0][0], -items[idx][0][1]))
        
        # compute representative key and count
        rep_key = items[rep_idx][0]
        if sum_counts:
            total = sum(items[idx][1] for idx in group_idxs)
            merged[rep_key] += total
        else:
            merged[rep_key] += items[rep_idx][1]
    
    return merged


In [4]:
pre_segments = get_combined_segments('raw/JM96_PRE__20250124_1204.bam')
sln_segments = get_combined_segments('raw/JM96_SLN__20251103_1409.bam')


In [5]:
pre_dels = detect_deletions(pre_segments)
pre_dels_merged = merge_blocks(pre_dels, 5)
print(len(pre_dels), len(pre_dels_merged))
pre_dels_merged.most_common(5)

1787 82


[((1084, 3594), 77986),
 ((7171, 9782), 18),
 ((6863, 7050), 15),
 ((3874, 4170), 7),
 ((3689, 4872), 7)]

In [7]:
sln_dels = detect_deletions(sln_segments)
sln_dels_merged = merge_blocks(sln_dels, 5)
print(len(sln_dels), len(sln_dels_merged))
sln_dels_merged.most_common(5)

1991 47


[((1075, 6368), 98700),
 ((6903, 7666), 545),
 ((832, 976), 474),
 ((599, 943), 6),
 ((1393, 6548), 5)]

In [9]:
import glob

all_hits = Counter()

for path in sorted(glob.glob('raw/*.bam')):
    segments = get_combined_segments(path)
    del_blocks = detect_deletions(segments)
    del_blocks_merged = merge_blocks(del_blocks, 5)
    print(path)
    print(del_blocks_merged.most_common(5))
    print()
    for block, _ in del_blocks_merged.most_common(5):
        all_hits[block] += 1
        
print(all_hits)

raw/JM96_DRG__20251218_1416.bam
[((1084, 6582), 37070), ((1057, 4956), 38), ((5757, 9791), 8), ((924, 3631), 4), ((5832, 8111), 2)]

raw/JM96_PBMC_20230406__20230601_1458.bam
[((1059, 4380), 292), ((3619, 5061), 61), ((1072, 2849), 8), ((1045, 1815), 5), ((1105, 1314), 4)]

raw/JM96_PBMC_20230406__20230706_1701.bam
[((1077, 4994), 849), ((715, 785), 1), ((757, 827), 1)]

raw/JM96_PBMC_20230406__20230719_1213.bam
[((1077, 4994), 72163), ((507, 567), 8), ((721, 816), 4), ((6353, 6418), 4), ((525, 584), 4)]

raw/JM96_PBMC_20230711__20230820_1323.bam
[((1081, 1312), 115), ((6415, 6506), 1), ((1728, 1781), 1), ((2115, 2177), 1), ((2224, 2339), 1)]

raw/JM96_PBMC_20230711__20230825_1500.bam
[((1081, 1312), 215), ((3320, 3720), 3), ((1060, 1842), 2), ((6567, 6622), 1), ((1003, 1430), 1)]

raw/JM96_PBMC_20230711__20230902_1607.bam
[((1079, 3716), 29), ((1089, 4334), 10), ((3701, 3866), 1), ((5370, 5482), 1)]

raw/JM96_PRE__20250124_1204.bam
[((1084, 3594), 77986), ((7171, 9782), 18), ((6863, 7

In [11]:
merge_blocks(all_hits, 5)

Counter({(1082, 3034): 233,
         (6903, 7666): 26,
         (1686, 1804): 11,
         (6580, 9120): 10,
         (1854, 2182): 9,
         (2264, 3323): 7,
         (924, 3631): 6,
         (599, 943): 6,
         (6117, 6533): 6,
         (6013, 6121): 5,
         (1178, 1768): 5,
         (5669, 5858): 4,
         (2311, 2466): 4,
         (1754, 5557): 4,
         (1222, 2413): 4,
         (1942, 2029): 4,
         (1393, 6548): 3,
         (996, 4849): 3,
         (832, 976): 3,
         (5906, 5959): 3,
         (6374, 6426): 3,
         (1967, 4711): 3,
         (5249, 5540): 3,
         (6184, 6418): 2,
         (525, 584): 2,
         (3700, 3801): 2,
         (7171, 9782): 2,
         (2772, 6107): 2,
         (2181, 2499): 2,
         (3014, 4471): 2,
         (6214, 6312): 2,
         (841, 957): 2,
         (3841, 3958): 2,
         (4623, 4782): 2,
         (6153, 6219): 2,
         (5853, 6045): 2,
         (4091, 4579): 2,
         (7741, 9321): 2,
         (1669, 5